# Lanuv: Excel into Pandas 

### Daten einlesen 

In [2]:
# Benötigte Pakete installieren 
import pandas as pd
import numpy as np
import json
import requests

In [3]:
# Excel einlesen  
f = 'Anfrage_Rentmeister.xlsx'
df = pd.read_excel(f, sheetname='NO2_JMW')


In [4]:
# Wie groß ist der Data Frame ?
df.shape

(153, 11)

In [5]:
# Welche Spaltennamen gibt es? 
df.columns
# df.sort_values(by='2014')

Index(['Stationscode', 'Stationsname', 'Schadstoff', 'Einheit', 'dataGroup', 2014, 2015, 'TYPE_OF_STATION', 'TYPE_OF_AREA', 'Länge', 'Breite'], dtype='object')

In [6]:
# Erste 5 Zeilen anzeigen 
df.head()

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,2014,2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite
0,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383
1,DENW006,Lünen-Niederaden,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.438,24.3654,Hintergrund,vorstädtisches Gebiet,7.569838,51.592206
2,DENW008,Dortmund-Eving,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.6639,26.5309,Hintergrund,städtisches Gebiet,7.457463,51.536911
3,DENW010,Unna-Königsborn,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,20.9525,20.9618,Hintergrund,vorstädtisches Gebiet,7.693835,51.547495
4,DENW021,Bottrop-Welheim,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.3434,26.3245,Industrie,städtisches Gebiet,6.976908,51.525963


### Daten bereinigen / umbauen / Spalten hinzufügen

In [7]:
# Städte rausziehen, Spalte Stadt hinzufügen

stadt = df['Stationsname'].str.split(' ').str.get(0)
stadt2 = stadt.str.split('-').str.get(0)

df['Stadt'] = stadt2
df['Stadt'].tail()

148    Düsseldorf
149     Paderborn
150       Bottrop
151      Schwerte
152         Herne
Name: Stadt, dtype: object

In [8]:
# 2014 und 2015 machen als Spaltenbeschriftung Probleme, deshalb werden die umbenannt
df.columns = ['Stationscode', 'Stationsname','Schadstoff', 'Einheit', 'dataGroup', 'Jahr-2014', 'Jahr-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', 'Länge', 'Breite', 'Stadt']

In [9]:
# Geklappt?
df.head(2)

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt
0,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383,Datteln
1,DENW006,Lünen-Niederaden,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,27.438,24.3654,Hintergrund,vorstädtisches Gebiet,7.569838,51.592206,Lünen


In [10]:
df.head(1)

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt
0,DENW002,Datteln-Hagem,Stickstoffdioxid,µg/m³,halb- oder stündliche Messung,21.1261,20.3173,Hintergrund,vorstädtisches Gebiet,7.331405,51.641383,Datteln


In [11]:
# Ausgeben, welche Datentypen in den Spalten sind. Bis auf Länge und Breite werden die Werte nicht als numerisch erkannt. 
df.dtypes

Stationscode        object
Stationsname        object
Schadstoff          object
Einheit             object
dataGroup           object
Jahr-2014           object
Jahr-2015           object
TYPE_OF_STATION     object
TYPE_OF_AREA        object
Länge              float64
Breite             float64
Stadt               object
dtype: object

In [12]:
# Liste mit Stationsnamen erzeugen für merge mit Rohdaten
stationsnamen = df[['Stationscode', 'Stationsname', 'Stadt', 'Länge', 'Breite']]
stationsnamen.head()

,Stationscode,Stationsname,Stadt,Länge,Breite
0,DENW002,Datteln-Hagem,Datteln,7.331405,51.641383
1,DENW006,Lünen-Niederaden,Lünen,7.569838,51.592206
2,DENW008,Dortmund-Eving,Dortmund,7.457463,51.536911
3,DENW010,Unna-Königsborn,Unna,7.693835,51.547495
4,DENW021,Bottrop-Welheim,Bottrop,6.976908,51.525963


In [13]:
pd.to_pickle(stationsnamen, './stationsnamen.pickle')

In [30]:
stationsnamen[stationsnamen.Stationscode == 'DENW136']

,Stationscode,Stationsname,Stadt
41,DENW136,Dortmund Brackeler Straße,Dortmund
42,DENW136,Dortmund Brackeler Straße,Dortmund


In [31]:
stationsnamen[stationsnamen.Stationscode == 'DENW082']

,Stationscode,Stationsname,Stadt
28,DENW082,Düsseldorf Corneliusstraße,Düsseldorf
29,DENW082,Düsseldorf Corneliusstraße,Düsseldorf


In [13]:
### Einige Zeilen enthalten für 2014 und 2015 seltsame oder keine Werte: '-' oder '-777.000'. 
# Pandas zählt diese Werte als Strings, deshalb der dtype 'object', weil Zahlen und Strings gemischt vorkommen. 

# Wie viele Bindestriche gibt es in den 2014er Werten? 
bindestrich_2014 = df[df['Jahr-2014']== '-']
bindestrich_2014.shape
# 2014 13 Stationen mit '-'

(13, 12)

In [14]:
# Wie viele Bindestriche gibt es in den 2015er Werten? 
bindestrich_2015 = df[df['Jahr-2015']== '-']
bindestrich_2015.shape

# 2015 11 Stationen mit '-'

(11, 12)

In [15]:
df[(df['Jahr-2014'] == '-') & (df['Jahr-2015'] == '-')]
# Keine Station 2014 und 2015 mit '-'

,Stationscode,Stationsname,Schadstoff,Einheit,dataGroup,Jahr-2014,Jahr-2015,TYPE_OF_STATION,TYPE_OF_AREA,Länge,Breite,Stadt


In [16]:
# Neuen DF clean_year erstellen
clean_year = df[['Jahr-2014', 'Jahr-2015']]

In [17]:
# Werte auf 'numerisch' setzen, wenn das nicht klappt, NAN eingeben
clean_year['Jahr-2014'] = pd.to_numeric(clean_year['Jahr-2014'], errors='coerce')

AttributeError: 'module' object has no attribute 'to_numeric'

In [ ]:
clean_year['Jahr-2015'] = pd.to_numeric(clean_year['Jahr-2015'], errors='coerce')
# clean_year.applymap(lambda x: isinstance(x, (int, float)))clean_year = pd.to_numeric(clean_year['Jahr-2015'], errors='coerce')
# clean_year.applymap(lambda x: isinstance(x, (int, float)))

In [ ]:
clean_year.dtypes

In [ ]:
# Ergebnis: Bindestriche sind weg, aber es bleiben Werte wie '-777.000'
clean_year.tail()

In [ ]:
# Werte kleiner 0 auf Nan setzen 
clean_year['Jahr-2014'] = clean_year['Jahr-2014'].where(clean_year['Jahr-2014'] > 0, np.nan)
clean_year['Jahr-2014'][100:110]

In [ ]:
clean_year['Jahr-2015'] = clean_year['Jahr-2015'].where(clean_year['Jahr-2015'] > 0, np.nan)
clean_year['Jahr-2015'][7]

In [ ]:
# saubere Spalten zum DF hinzufügen 
df['c-2014'] = clean_year['Jahr-2014']
df['c-2015'] = clean_year['Jahr-2015']

In [ ]:
# Für die App ist eine Spalte 'id' nötig, die Zeilen durchzählt:
df['id'] = df.index

In [ ]:
# Neuen DF mit ausgewählten Spalten erstellen
no2 = df[['Stationsname', 'Stadt', 'Schadstoff', 'Einheit', 'c-2014', 'c-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', 'Länge', 'Breite', 'id']]
no2.columns

In [ ]:
# Geklappt? 
no2.head()

In [ ]:
# Bezeichnung für 2014 und 2015 noch mal ändern 
no2.columns = ['name', 'stadt', 'stoff', 'einheit', 'jahr_2014', 'jahr_2015', 'typ','gebiet', 'x', 'y', 'id']
no2 = no2.sort_values(by='jahr_2015', ascending=False)
no2.tail(20)

In [ ]:
# Als CSV rausgeben
### no2.to_csv('no2_c.csv')
# Obacht: Wenn Dateiname nicht ge#ndert wird, wird Datei immer wieder überschrieben

In [ ]:
### _Ab hier sammele ich Codes, die ich geschrieben, aber dann doch nicht verwendet habe, 
# die aber vielleicht noch nützlich sein können:_

# Spalten rausschmeißen: 
# df = df.drop(['Stationscode','Stationsname', 'Schadstoff', 'Einheit', 'dataGroup', 'Jahr-2014', 'Jahr-2015', 'TYPE_OF_STATION', 'TYPE_OF_AREA', ] , axis=1)
# df = df.drop(['year_2014','year_2015'], axis=1)

# Index auf Stationsname setzen 
# df.index = df['Stationsname']

### Informationen zu Grenzwerten hinzufügen 



In [ ]:
# Grenzwert 40 überschritten in 2015?

drueber = no2[no2['jahr_2015'] > 40]
drueber.shape
drueber.head(1)

# 2014 wurde der Grenzwert an 64 Messstationen überschritten 

In [ ]:
# Welches sind die Stationen mit den höchsten Messwerten?
drueber_s = drueber.sort_values(by='jahr_2015', ascending = False)
drueber_s.head(1)

In [ ]:
# 2015: Welche Städte hatten die meisten überschrittenen Grenzwerte?
top_2015 = df.sort_values(by='c-2015', ascending=False)
drueber_2015 = top_2015[top_2015['c-2015'] >40]
drueber_2015_staedte = drueber_2015.groupby('Stadt').agg(['count'])
drueber_2015_staedte.sort_values(by=[('Stationsname','count')], ascending=False).head(5)

In [ ]:
# Wie viele Messstellen wo? 

mess = df.groupby('Stadt').agg(['count'])
mess.sort_values(by=[('Stationsname','count')], ascending=False).head(5)

In [ ]:
# Next step: Anteil der Stationen mit überschrittenem Grenzwert an der Anzahl der Stationen berechnen

In [ ]:
# Spalte 'ueberschritten' hinzufügen
no2.head()
no2['ueberschritten'] = no2['jahr_2015'] > 40
test = no2[no2['ueberschritten'] == True]
test.shape

In [ ]:
no2.describe()

Unterschied 2014 und 2015 berechnen 
http://chrisalbon.com/python/pandas_make_new_columns_using_functions.html

In [ ]:
# Create a function that takes two inputs, pre and post
def pre_post_difference(pre, post):
    # returns the difference between post and pre
    return post - pre

In [ ]:
# Create a variable that is the output of the function
no2['change'] = pre_post_difference(no2['jahr_2014'], no2['jahr_2015'])

# View the dataframe
change = no2[no2['change'] > 0]

In [ ]:
change.shape

In [ ]:
# In 20 von ca 130 Messstationen sind Werte gestiegen 
20/130

In [ ]:
### Output für FNA und Bubble Chart vorbereiten 
no2.head(2)

### File ausgeben: 

In [ ]:
no2.to_csv('no2_marker.csv')

In [ ]:
change_s = no2.sort_values(by='change', ascending=False)

### Bubble Chart ! 

- On the x-axis: 2014 - (“income”)
- On the y-axis: 2015 - (“health”). 
- Size of the bubbles: Change - (population) 
- Some tools call that a scatterplot, some call it a bubble chart.

#### Matplotlib

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

#read data
data = pd.read_csv("no2_marker.csv")

#plot chart
# plt.scatter(np.log(no2['jahr_2015']), no2['jahr_2014'], s=no2['change']*200, c='blue')
# plt.xlim(xmin=0) #set origin for x axis to zero
# plt.show()

matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)

plt.scatter(no2['jahr_2014'], no2['jahr_2015'], s=70)
plt.title('2014 > 2015', loc='left', size=25)
plt.xlabel('2014', size=18)
plt.ylabel('2015', size=18)

x = range(0,70)
# plt.xticks(x)
# plt.yticks(x)

In [ ]:
# Bubble chart mit s=change bedeutet, dass alle Werte unter 0 nicht angezeigt werden
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
plt.scatter(no2['jahr_2014'], no2['jahr_2015'], s=no2['change']*700)

#### Seaborn

In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#read data
data = pd.read_csv("no2_marker.csv")

#plot chart
g = sns.regplot('jahr_2015', 'jahr_2014', data=data, color='r',fit_reg=False)
# g.set_xscale('log')
name="x_var"
plt.show()

#### Bokeh

In [ ]:
#import libraries
import pandas as pd
from bokeh.plotting import figure, show, output_file

#read data
data = pd.read_csv("no2_marker.csv")

#plot chart
p = figure()
p.scatter(no2['jahr_2014'], no2['jahr_2015'], radius=no2['change'],
          fill_color='blue', fill_alpha=0.4, line_color=None)

#write as html file and open in browser
output_file("scatterplot.html")
show(p)

